# LangChain Agents: A Deep Dive

**Objective:** This notebook provides a detailed walkthrough of building powerful, single agents using LangChain. While the course's Day 5 lab introduced the basics, this session focuses on the internal mechanics, advanced tool creation, and state management to give you a deeper understanding of how to build and debug robust LangChain agents.

**Target Audience:** Software engineers attending the AI-Driven Software Engineering Program.

**Core Concepts Covered:**
1.  **Review of Agent Fundamentals:** A quick recap of the `AgentExecutor` and the ReAct (Reason+Act) loop.
2.  **Custom Tools in Detail:** Mastering the `@tool` decorator and understanding why a good docstring is the most critical part of a tool.
3.  **Agent State & Memory:** Giving your agent conversational memory to handle multi-turn interactions.
4.  **Debugging Agent Behavior:** Using `verbose=True` to interpret the agent's thought process and fix issues.

In [12]:
import sys
import os

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact, clean_llm_output

client, model_name, api_provider = setup_llm_client(model_name="gpt-4.1")

# Load the SQL schema from Day 2
sql_schema = load_artifact("artifacts/schema.sql")
if not sql_schema:
    print("Warning: Could not load schema.sql. Lab may not function correctly.")

2025-09-26 15:54:15,421 ag_aisoftdev.utils INFO LLM Client configured provider=openai model=gpt-4.1 latency_ms=None artifacts_path=None


In [16]:
# Challenge 1: Generate FastAPI app with in-memory data storage
in_memory_api_prompt = """
You are a senior Python backend developer. Using the following code as context, generate a single Python script for a FastAPI application.
Code:
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain import hub
from langchain.agents import create_tool_calling_agent, AgentExecutor

llm = ChatOpenAI(model="gpt-4o")
tools = [TavilySearchResults(max_results=2)]
prompt = hub.pull("hwchase17/openai-functions-agent")
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

print("--- Recap: Agent with Web Search ---")
agent_executor.invoke({"input": "Briefly compare the creation myths in Greek, Egyptian, Yoruba, and Roman mythology."})

Requirements:
1. Create Pydantic models for API data validation, matching the structure of the 'users' table.
2. Implement FastAPI endpoints for the invoke method of the agent_executor.
4. The output should be a single, clean Python script named 'mythology_agent.py' that can be run directly.
5. Include all necessary imports and FastAPI setup code.
6. Ensure endpoints follow RESTful conventions and return appropriate status codes.
7. Add comments to explain each major section of the code.

Only output the Python code, without markdown fences or extra explanation.
"""

print("--- Generating FastAPI app with in-memory database ---")
if sql_schema:
    generated_api_code = get_completion(in_memory_api_prompt, client, model_name, api_provider)
    cleaned_code = clean_llm_output(generated_api_code, language='python')
    print(cleaned_code)
    save_artifact(cleaned_code, "app/mythology_agent.py", overwrite=True)
else:
    print("Skipping API generation because schema is missing.")

--- Generating FastAPI app with in-memory database ---
# mythology_agent.py

from fastapi import FastAPI, HTTPException, status
from pydantic import BaseModel, EmailStr, Field
from typing import Optional
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain import hub
from langchain.agents import create_tool_calling_agent, AgentExecutor

# -------------------------------
# Pydantic Models
# Assuming 'users' table has: id, name, email, is_active, created_at
# -------------------------------
class UserBase(BaseModel):
    name: str = Field(..., example="John Doe")
    email: EmailStr = Field(..., example="john@example.com")
    is_active: bool = Field(default=True)

class UserCreate(UserBase):
    password: str = Field(..., example="secret123")

class User(UserBase):
    id: int
    created_at: Optional[str]

    class Config:
        orm_mode = True

# -------------------------------
# FastAPI App Initialization
# 

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain import hub
from langchain.agents import create_tool_calling_agent, AgentExecutor

llm = ChatOpenAI(model="gpt-4o")
tools = [TavilySearchResults(max_results=2)]
prompt = hub.pull("hwchase17/openai-functions-agent")
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

print("--- Recap: Agent with Web Search ---")
agent_executor.invoke({"input": "Briefly compare the creation myths in Greek, Egyptian, Yoruba, and Roman mythology."})

C:\Users\labadmin\AppData\Local\Temp\2\ipykernel_11524\1973698611.py:7: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tools = [TavilySearchResults(max_results=2)]


--- Recap: Agent with Web Search ---


> Entering new AgentExecutor chain...
Creation myths from different cultures provide insights into how ancient peoples understood the origins of the world and their place within it. Here's a brief comparison of creation myths from Greek, Egyptian, Yoruba, and Roman mythology:

1. **Greek Mythology**:
   - Begins with Chaos, a formless void.
   - From Chaos emerged Gaia (Earth), Tartarus (the Underworld), and Eros (Love).
   - Gaia gave birth to Uranus (Sky), and together they produced the Titans, the Cyclopes, and the Hecatoncheires.
   - The overthrow of Uranus by his son Cronus, and later Cronus by Zeus, marks key transitions in the Greek mythological cosmos.

2. **Egyptian Mythology**:
   - Different creation myths exist, reflecting the diversity of early Egyptian culture. Commonly, the Heliopolitan myth is referenced.
   - The world begins as a primordial ocean called Nun.
   - Atum, the first god, emerged from Nun and created Shu (air) and Te

{'input': 'Briefly compare the creation myths in Greek, Egyptian, Yoruba, and Roman mythology.',
 'output': "Creation myths from different cultures provide insights into how ancient peoples understood the origins of the world and their place within it. Here's a brief comparison of creation myths from Greek, Egyptian, Yoruba, and Roman mythology:\n\n1. **Greek Mythology**:\n   - Begins with Chaos, a formless void.\n   - From Chaos emerged Gaia (Earth), Tartarus (the Underworld), and Eros (Love).\n   - Gaia gave birth to Uranus (Sky), and together they produced the Titans, the Cyclopes, and the Hecatoncheires.\n   - The overthrow of Uranus by his son Cronus, and later Cronus by Zeus, marks key transitions in the Greek mythological cosmos.\n\n2. **Egyptian Mythology**:\n   - Different creation myths exist, reflecting the diversity of early Egyptian culture. Commonly, the Heliopolitan myth is referenced.\n   - The world begins as a primordial ocean called Nun.\n   - Atum, the first god, em

## 1. Setup

First, we'll install the necessary libraries and set up our environment by loading the required API keys.

In [5]:
import os
from dotenv import load_dotenv

load_dotenv()

if not os.getenv("OPENAI_API_KEY") or not os.getenv("TAVILY_API_KEY"):
    print("ERROR: API keys not found. Please check your .env file.")

try:
    import ddgs
except ImportError:
    print("ERROR: DuckDuckGo search package 'ddgs' not found. Please install it with 'pip install ddgs'.")

## 2. Review of Agent Fundamentals

Let's quickly recap the core components of a LangChain agent from the Day 5 lab. An agent consists of:

- **LLM:** The language model that acts as the agent's "brain."
- **Tools:** Functions the agent can call to interact with the outside world (e.g., search, calculators, file readers).
- **Prompt Template:** A template that structures the input to the LLM, telling it its persona, the tools it has, and the user's request. It critically includes an `agent_scratchpad` for the ReAct loop.
- **Agent Executor:** The runtime environment that orchestrates the **Thought -> Action -> Observation** loop.

In [7]:
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain import hub
from langchain.agents import create_tool_calling_agent, AgentExecutor

llm = ChatOpenAI(model="gpt-4o")
tools = [TavilySearchResults(max_results=2)]
prompt = hub.pull("hwchase17/openai-functions-agent")
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

print("--- Recap: Agent with Web Search ---")
agent_executor.invoke({"input": "Briefly compare the creation myths in Greek, Egyptian, Yoruba, and Roman mythology."})

C:\Users\labadmin\AppData\Local\Temp\2\ipykernel_11524\1973698611.py:7: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tools = [TavilySearchResults(max_results=2)]


--- Recap: Agent with Web Search ---


> Entering new AgentExecutor chain...
Creation myths from different cultures provide insights into how ancient peoples understood the origins of the world and their place within it. Here's a brief comparison of creation myths from Greek, Egyptian, Yoruba, and Roman mythology:

1. **Greek Mythology**:
   - Begins with Chaos, a formless void.
   - From Chaos emerged Gaia (Earth), Tartarus (the Underworld), and Eros (Love).
   - Gaia gave birth to Uranus (Sky), and together they produced the Titans, the Cyclopes, and the Hecatoncheires.
   - The overthrow of Uranus by his son Cronus, and later Cronus by Zeus, marks key transitions in the Greek mythological cosmos.

2. **Egyptian Mythology**:
   - Different creation myths exist, reflecting the diversity of early Egyptian culture. Commonly, the Heliopolitan myth is referenced.
   - The world begins as a primordial ocean called Nun.
   - Atum, the first god, emerged from Nun and created Shu (air) and Te

{'input': 'Briefly compare the creation myths in Greek, Egyptian, Yoruba, and Roman mythology.',
 'output': "Creation myths from different cultures provide insights into how ancient peoples understood the origins of the world and their place within it. Here's a brief comparison of creation myths from Greek, Egyptian, Yoruba, and Roman mythology:\n\n1. **Greek Mythology**:\n   - Begins with Chaos, a formless void.\n   - From Chaos emerged Gaia (Earth), Tartarus (the Underworld), and Eros (Love).\n   - Gaia gave birth to Uranus (Sky), and together they produced the Titans, the Cyclopes, and the Hecatoncheires.\n   - The overthrow of Uranus by his son Cronus, and later Cronus by Zeus, marks key transitions in the Greek mythological cosmos.\n\n2. **Egyptian Mythology**:\n   - Different creation myths exist, reflecting the diversity of early Egyptian culture. Commonly, the Heliopolitan myth is referenced.\n   - The world begins as a primordial ocean called Nun.\n   - Atum, the first god, em

## 3. Custom Tools in Detail
The real power of LangChain agents comes from giving them custom tools to interact with your specific data or APIs. The `@tool` decorator is the easiest way to do this.

**The Golden Rule of Tools:** The agent **does not see your Python code**. It only sees the function name, its arguments (and their type hints), and the **docstring**. A clear, descriptive docstring is the most important piece of prompt engineering you can do for an agent.

Below, we define a custom tool called `compare_myths` that allows the agent to compare mythology themes across cultures using DuckDuckGo search. The agent can invoke this tool to answer questions like:
- "Compare underworld myths"
- "Compare hero myths"

In [8]:
@tool
def compare_myths(theme: str) -> str:
    """Research and compare how Greek, Egyptian, Yoruba, and Roman mythology interpret a given theme (e.g., creation, underworld, hero). Returns a summary of professionally researched findings from DuckDuckGo search results for each culture."""
    results = {}
    cultures = ["Greek", "Egyptian", "Yoruba", "Roman"]
    for culture in cultures:
        query = f"{culture} mythology {theme} myth"
        with DDGS() as ddg:
            search_results = list(ddg.text(query, max_results=2))
            summaries = [r.get('body', '') for r in search_results]
            results[culture] = " ".join(summaries) if summaries else "No results found."
    output = ""
    for culture, summary in results.items():
        output += f"\n--- {culture} Mythology ---\n{summary}\n"
    return output

tools = [compare_myths]
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Custom tool usage examples for mythology comparison
print("--- Compare Underworld Myths ---")
result_underworld = agent_executor.invoke({"input": "Compare underworld myths"})
print(result_underworld['output'])

print("--- Compare Hero Myths ---")
result_hero = agent_executor.invoke({"input": "Compare hero myths"})
print(result_hero['output'])

--- Compare Underworld Myths ---


> Entering new AgentExecutor chain...

Invoking: `compare_myths` with `{'theme': 'underworld'}`



--- Greek Mythology ---
In Greek mythology , the underworld or Hades (Ancient Greek : ᾍδης, romanized: Háidēs) is a distinct realm (one of the three realms that make up the cosmos) where an individual goes after death. The earliest idea of afterlife in Greek myth is that, at the moment of death, an individual's essence (psyche) is separated from the corpse and transported to the underworld . [1] In early ... Circling the Underworld seven times, Styx was the river of hatred and unbreakable oaths; the gods are often depicted as taking vows by its waters. • The Acheron. The river of sorrow and pain, black and deep. • The Cocytus. The river of lamentation and wailing. • The Phlegethon. The river of fire, possibly leading to the depths of Tartarus.

--- Egyptian Mythology ---
The walls of many Pharaonic tombs in the Valley of the Kings are decorated with the 

## 4. Agent State & Memory

By default, the `AgentExecutor` is stateless. Each call is independent. To have a conversation, we need to add a memory component. The easiest way is to include placeholders for chat history in the prompt and manage the history manually.

LangChain's Expression Language (LCEL) makes this more manageable.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage

# 1. Create a new prompt that includes a placeholder for chat history
memory_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an agent that researches and specializes in mythology. You compare mythology themes across Greek, Egyptian, Yoruba, and Roman cultures."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

# 2. Create the agent with the new prompt
memory_tools = [compare_myths]
memory_agent = create_tool_calling_agent(llm, memory_tools, memory_prompt)
memory_executor = AgentExecutor(agent=memory_agent, tools=memory_tools, verbose=True)

# 3. Manage the chat history manually
chat_history = []

print("--- Conversation Turn 1 ---")
input1 = "Compare creation myths."
result1 = memory_executor.invoke({"input": input1, "chat_history": chat_history})
chat_history.extend([
    HumanMessage(content=input1),
    AIMessage(content=result1["output"]),
])
print(f"\nAI: {result1['output']}")

print("\n--- Conversation Turn 2 (with memory) ---")
input2 = "Now compare underworld myths."
result2 = memory_executor.invoke({"input": input2, "chat_history": chat_history})
print(f"\nAI: {result2['output']}")

--- Conversation Turn 1 ---


> Entering new AgentExecutor chain...

Invoking: `compare_myths` with `{'theme': 'creation in Greek mythology'}`



Invoking: `compare_myths` with `{'theme': 'creation in Greek mythology'}`



--- Greek Mythology ---
Like most, if not all peoples of the world, the Greek have their own myths and legends about how the world came to be. While there are a few varying versions of the Greek Creation Myth , they all tend to revolve around a number of magical deities that just couldn't get along for some reason. This version of the Greek Creation Myth is heavily borrowed from the works of a seventh-century BCE ... The Creation Myths of Greece: How the World Came to Be I. Introduction to Greek Creation Myths Greek mythology is a rich tapestry of stories, beliefs, and traditions that formed the foundation of ancient Greek culture. It encompasses a wide range of deities, heroes, and narratives that explain the world, human existence, and the cosmos.

--- Egyptian Myt

In [11]:
type (result2)
result2.keys()

dict_keys(['input', 'chat_history', 'output'])

In [18]:
# Prompt for LLM to generate requirements.txt
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain import hub
from langchain.agents import create_tool_calling_agent, AgentExecutor

llm = ChatOpenAI(model="gpt-4o")
tools = [TavilySearchResults(max_results=2)]
prompt = hub.pull("hwchase17/openai-functions-agent")
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

app_code = '''
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain import hub
from langchain.agents import create_tool_calling_agent, AgentExecutor

llm = ChatOpenAI(model=\"gpt-4o\")
tools = [TavilySearchResults(max_results=2)]
prompt = hub.pull(\"hwchase17/openai-functions-agent\")
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

print(\"--- Recap: Agent with Web Search ---\")
agent_executor.invoke({\"input\": \"Briefly compare the creation myths in Greek, Egyptian, Yoruba, and Roman mythology.\"})
'''

requirements_prompt = f"""
Analyze the following FastAPI application code and generate a requirements.txt file.
Include all external dependencies found in the import statements (e.g., fastapi, uvicorn, langchain, openai, ddgs).
Also include pytest for testing.

Code:
{app_code}

Output only the requirements.txt content.
"""

# Use your LLM utility to get the requirements
requirements_txt = get_completion(requirements_prompt, client, model_name, api_provider)
with open("requirements.txt", "w") as f:
    f.write(requirements_txt)

In [20]:
dockerfile_prompt = """
Generate a multi-stage Dockerfile for a Python FastAPI application.
Requirements:
- Use python:3.11-slim as the base image.
- The first stage should install dependencies from requirements.txt.
- The final stage should copy the installed dependencies and the application code.
- The CMD should run the app using uvicorn.

(no markdown fences)

Output only the Dockerfile content.
"""

dockerfile_content = get_completion(dockerfile_prompt, client, model_name, api_provider)
with open("Dockerfile", "w") as f:
    f.write(dockerfile_content)

In [23]:
ci_prompt = """
Generate a GitHub Actions workflow file named ci.yml.
Requirements:
- Trigger on any push to the main branch.
- Define a job build-and-test that runs on ubuntu-latest.
- Steps: 1) Check out code, 2) Set up Python, 3) Install dependencies from requirements.txt, 4) Run pytest.

(no markdown fences)
Output only the YAML content.
"""

ci_yaml = get_completion(ci_prompt, client, model_name, api_provider)
os.makedirs(".github/workflows", exist_ok=True)
with open(".github/workflows/ci.yml", "w") as f:
    f.write(ci_yaml)

## 5. Debugging Agent Behavior

The most important tool for debugging an agent is `verbose=True` in the `AgentExecutor`. This prints out the agent's internal monologue, showing you the **Thought -> Action -> Observation** loop in real-time.

When an agent fails, look at the verbose output:
-   **Thought:** What was the agent thinking? Did it misunderstand the goal?
-   **Action:** Which tool did it choose? Were the parameters correct? If it didn't choose a tool when it should have, the tool's docstring might be unclear.
-   **Observation:** What was the output of the tool? Was it an error? Was the format unexpected?

Most agent failures can be traced back to a poorly worded prompt or an unclear tool docstring.

## Lab Conclusion

In this deep dive, you've reinforced your understanding of LangChain agents. You've seen how to create powerful, specialized agents by giving them custom tools and how to add memory for conversational context. Most importantly, you've learned how to debug agentic behavior by analyzing the ReAct loop.

**Key Takeaways:**
- An agent's capabilities are defined by its tools.
- A tool's docstring is its most important feature.
- Conversational memory requires explicitly managing and passing chat history.
- `verbose=True` is your best friend for debugging.